In [ ]:
from util import *
import time
import datetime

import cPickle as pickle
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pylab
import re
import scipy as sp
import seaborn
import sklearn.feature_extraction.text as text

from gensim import corpora, models
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn import decomposition

In [ ]:
lines=[]
with open("data/reviews_full.dat", "r") as fh:
    lines = fh.readlines()  

In [ ]:
userid = []
rating = []
docs = []
business = []
i = 0
j = 0
error_line_num = []
error_lines = []
for line in lines:
    try:
        i = i + 1
        l = line.split('\t', 4)
        userid.append(l[0])
        business.append(l[1])
        rating.append(l[2])
        docs.append(l[3])
        #d = clean(l[3])
        #kmers = getKmers(d)
        #d.extend(kmers)
        #docs.append(d)
    except Exception as e:
        j = j + 1
        error_line_num.append(i)
        error_lines.append(line)

print 'Training Data: Number of lines processed: ' + str(i)
print 'Training Data: Length of userid array: ' + str(len(userid))
print 'Training Data: Length of rating array: ' + str(len(rating))
print 'Training Data: Length of docs array: ' + str(len(docs))
print 'Training Data: Length of business array: ' + str(len(business))
print 'Training Data: Number of exceptions encountered: ' + str(j)



In [ ]:

from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
n_samples = 50000
n_features = 15000
n_components = 30
n_top_words = 50

data_samples = docs[:n_samples]

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))


# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)

tfidf_nmf = nmf.transform(tfidf)

print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
#tfidf_feature_names = tfidf_vectorizer.get_feature_names()
vocab = np.array(tfidf_vectorizer.get_feature_names())
#print_top_words(nmf, tfidf_feature_names, n_top_words)




In [1]:
from util import *
import time
import datetime

import cPickle as pickle
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pylab
import re
import scipy as sp
import seaborn
import sklearn.feature_extraction.text as text

from gensim import corpora, models
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn import decomposition

lines=[]
with open("data/reviews_full.dat", "r") as fh:
    lines = fh.readlines()  
    
userid = []
rating = []
docs = []
business = []
i = 0
j = 0
error_line_num = []
error_lines = []
for line in lines:
    try:
        i = i + 1
        l = line.split('\t', 4)
        userid.append(l[0])
        business.append(l[1])
        rating.append(l[2])
        docs.append(l[3])
        #d = clean(l[3])
        #kmers = getKmers(d)
        #d.extend(kmers)
        #docs.append(d)
    except Exception as e:
        j = j + 1
        error_line_num.append(i)
        error_lines.append(line)

print 'Training Data: Number of lines processed: ' + str(i)
print 'Training Data: Length of userid array: ' + str(len(userid))
print 'Training Data: Length of rating array: ' + str(len(rating))
print 'Training Data: Length of docs array: ' + str(len(docs))
print 'Training Data: Length of business array: ' + str(len(business))
print 'Training Data: Number of exceptions encountered: ' + str(j)

Training Data: Number of lines processed: 317586
Training Data: Length of userid array: 317586
Training Data: Length of rating array: 317586
Training Data: Length of docs array: 317586
Training Data: Length of business array: 317586
Training Data: Number of exceptions encountered: 0


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.neighbors import KNeighborsRegressor

total_data =50000
train_data =40000

def getSentiment(x):
    if x < 3.0:
        return 0
    else:
        return 1
    

def nmf(rating):
    nmf = load_pickle('nmf.pickle')
    print nmf.shape
    trainTopics= nmf[:total_data]
    rating=rating[:total_data]
    print trainTopics.shape
    print len(rating)
    trainTopics = trainTopics / np.sum(trainTopics, axis=1, keepdims=True)
    d, f = trainTopics.shape
    cols = ["Topic"+str(i) for i in xrange(1, f+1)]
    nmfDF = pd.DataFrame(trainTopics, columns=cols)
    nmfDF['rating'] = map(float,rating)
    cols = [u'Topic1', u'Topic2', u'Topic3', u'Topic4', u'Topic5', u'Topic6', u'Topic7', u'Topic8', u'Topic9', u'Topic10', u'Topic11', u'Topic12', u'Topic13', u'Topic14', u'Topic15', u'Topic16', u'Topic17', u'Topic18', u'Topic19', u'Topic20']
    Xtrain = nmfDF[:train_data][cols]
    Ytrain = nmfDF[:train_data]['rating']
    Xtest = nmfDF[train_data:][cols]
    Ytest = nmfDF[train_data:]['rating']
    
    return nmfDF, Xtrain, Ytrain, Xtest, Ytest
    
    
def nmf_sentiment(rating):
    nmfDF,a,b,c,d = nmf(rating)
    nmfDF['Sentiment'] = nmfDF['rating'].map(getSentiment)
    nmfDF = nmfDF.dropna(how='any')
    cols = [u'Topic1', u'Topic2', u'Topic3', u'Topic4', u'Topic5', u'Topic6', u'Topic7', u'Topic8', u'Topic9', u'Topic10', u'Topic11', u'Topic12', u'Topic13', u'Topic14', u'Topic15', u'Topic16', u'Topic17', u'Topic18', u'Topic19', u'Topic20', u'Sentiment']
    Xtrain = nmfDF[:train_data][cols]
    Ytrain = nmfDF[:train_data]['rating']
    Xtest = nmfDF[train_data:][cols]
    Ytest = nmfDF[train_data:]['rating']
    
    return nmfDF,Xtrain,Ytrain,Xtest,Ytest

    
    
def lda_sentiment(rating):
    lda = load_pickle('lda.pickle')
    trainTopics= lda[:total_data]
    rating=rating[:total_data]
    trainTopics = trainTopics / np.sum(trainTopics, axis=1, keepdims=True)
    d, f = trainTopics.shape
    cols = ["Topic"+str(i) for i in xrange(1, f+1)]
    ldaDF = pd.DataFrame(trainTopics, columns=cols)
    ldaDF['rating'] = map(float,rating)
    ldaDF['Sentiment'] = ldaDF['rating'].map(getSentiment)
    ldaDF = ldaDF.dropna(how='any')
    cols = [u'Topic1', u'Topic2', u'Topic3', u'Topic4', u'Topic5', u'Topic6', u'Topic7', u'Topic8', u'Topic9', u'Topic10', u'Topic11', u'Topic12', u'Topic13', u'Topic14', u'Topic15', u'Topic16', u'Topic17', u'Topic18', u'Topic19', u'Topic20', u'Sentiment']
    Xtrain = ldaDF[:train_data][cols]
    Ytrain = ldaDF[:train_data]['rating']
    Xtest = ldaDF[train_data:][cols]
    Ytest = ldaDF[train_data:]['rating']
    
    return ldaDF,Xtrain,Ytrain,Xtest,Ytest
    


In [3]:
rating = map(float,rating)
print len(rating)
df,train_X,train_Y,test_X,test_Y = nmf(rating)


317586
(50000, 20)
(50000, 20)
50000


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.kernel_ridge import KernelRidge


from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score



ModelEvaluationResults = {}

rating = map(float,rating)

models= [nmf(rating),nmf_sentiment(rating),lda_sentiment(rating)]
model_names = ["NMF","NMF-Sentiment","LDA-Sentiment"]


results = {}

for (m,models_) in enumerate(models):
    df,train_X,train_Y,test_X,test_Y = models_
    print "Evalutaion of Model:  "+str(model_names[m])
    
    clfs = [ LogisticRegression(),
    KNeighborsRegressor(n_neighbors=3),
    SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='auto',kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
    ]
    #KernelRidge(alpha=1.0, coef0=1, degree=3, gamma=None, kernel='linear',kernel_params=None)]

    #clf_names = ['Logistic Regression','KNeighborsRegressor','SVR','KernelRidge']
    clf_names = ['Logistic Regression','KNeighborsRegressor','SVR']


    
    ClassifierEvaluationResults = {}
    
    for (i, clf_) in enumerate(clfs):
        #clf = clf_.fit(Xtrain, Ytrain)
        #preds = clf_.predict(Xtest)
        print "********************************************************************"
        print "Training validations evaluations for : "+str(clf_names[i])
        scores = cross_val_score(clf_, train_X, train_Y, cv=10)
        #predicted_ratings = cross_val_predict(clf_, test_X,test_Y, cv=10)
        print scores
        train_mean_score=scores.mean()
        print "Testing validations evaluations for : "+str(clf_names[i])


#         r2 = r2_score(test_Y, predicted_ratings, multioutput='uniform_average')
#         mae = median_absolute_error(test_Y, predicted_ratings)
#         msle =mean_squared_log_error(test_Y, predicted_ratings) 
#         mse = mean_squared_error(test_Y, predicted_ratings)
#         mae = mean_absolute_error(test_Y, predicted_ratings)
#         evs = explained_variance_score(test_Y, predicted_ratings)  
        
#         data ={'training score':train_mean_score,
#             'r2 score':r2,
#             'median absolute error':mae,
#             'mean squared log error':msle,
#             'mean sqaured error':mse,
#             'mean absolute error':mae,
#             'explained variance error':evs}

        data ={'training_score':train_mean_score}        
        
        ClassifierEvaluationResults[clf_names[i]] = data
        #print data
    
    ModelEvaluationResults[model_names[m]]=ClassifierEvaluationResults
    print ClassifierEvaluationResults
    print "********************************************************************"




(50000, 20)
(50000, 20)
50000
(50000, 20)
(50000, 20)
50000
Evalutaion of Model:  NMF
********************************************************************
Training validations evaluations for : Logistic Regression
[ 0.45127436  0.45938515  0.45388653  0.45613597  0.45013747  0.43375
  0.437       0.45047524  0.45472736  0.44947474]
Testing validations evaluations for : Logistic Regression
********************************************************************
Training validations evaluations for : KNeighborsRegressor
[ 0.04491631  0.0483624   0.00304631 -0.00614167  0.06293293  0.01400524
  0.05819278  0.03223458  0.03774928 -0.01082228]
Testing validations evaluations for : KNeighborsRegressor
********************************************************************
Training validations evaluations for : SVR


In [12]:


cols = ['training score', 'r2 score', 'median absolute error', 'mean squared log error','mean sqaured error','mean absolute error','explained variance error']
#cols = ['training score', 'r2 score', 'median absolute error', 'mean squared log error','mean sqaured error','mean absolute error','explained variance error']
clf_names = ['Logistic Regression','KNeighborsRegressor','SVR']
model_names = ["NMF","NMF-Sentiment","LDA-Sentiment"]




print ModelEvaluationResults

for m in model_names:
    
    print pd.DataFrame(m).T[clf_names].T

{'NMF': {'NMFSVR': {'r2 score': -0.20452192961501825, 'mean absolute error': 0.68470868630884918, 'mean sqaured error': 1.0720245173573664, 'training score': -0.19939618622008298, 'median absolute error': 0.68470868630884918, 'explained variance error': -0.19325356658519999, 'mean squared log error': 0.068210505250365788}, 'NMFLogistic Regression': {'r2 score': -0.011235955056179581, 'mean absolute error': 0.59999999999999998, 'mean sqaured error': 0.90000000000000002, 'training score': 0.37202380952380942, 'median absolute error': 0.59999999999999998, 'explained variance error': -2.2204460492503131e-16, 'mean squared log error': 0.060248331682490011}, 'NMFKNeighborsRegressor': {'r2 score': -0.24219725343320797, 'mean absolute error': 0.81666666666666665, 'mean sqaured error': 1.1055555555555554, 'training score': -0.81793112293368497, 'median absolute error': 0.81666666666666665, 'explained variance error': -0.20443196004993713, 'mean squared log error': 0.067274170527681892}}, 'LDA-S

ValueError: DataFrame constructor not properly called!